In [1]:
import requests, re, os, time, pickle
import pandas as pd

In [2]:
keywords = '''artificial intelligence'''
APIkey = ''

In [3]:
current_time = time.asctime()
current_time = current_time.replace("  ","_")
current_time = current_time.replace(" ","_")
current_time = current_time.replace(":","_")

In [4]:
os.popen("mkdir "+current_time)

In [5]:
def extract_metadata(keywords,APIkey):
    textdata =pd.DataFrame()
    start = 1
    count =100
    idx = 1
    while True:
        url = 'http://api.elsevier.com/content/search/index:SCIDIR?query='+keywords+'&count='+str(count)+"&start="+str(start)
        temp_df,num_results = extract_article_info(url,start,APIkey)
        if num_results == -1:
            break
        textdata=textdata.append(temp_df,ignore_index=True)
        if num_results<count:
            print("last page")
            break
        else:
            start = start+ 100 
        break
            
    return textdata

In [6]:
def extract_article_info(url,start,APIkey):
    intextdata =pd.DataFrame()
    resp = requests.get(url,headers={'Accept':'application/json','X-ELS-APIKey':APIkey})
    results = resp.json()
    num_results = 0
    identifier_set = []
    
    try:
        num_results = len(results['search-results']['entry'])
        for idx in range(0,num_results):
            try:
                identifier_set.append(results['search-results']['entry'][idx]['dc:identifier'])
            except:
                pass
        print(round(100*len(identifier_set)/num_results,2),'% of results from',start,"~",start+num_results-1,'can be accessed')
    except:
        print("No entry is found, you may need to change the keywords")
        return -1,-1
    idx=0
    for identifier in identifier_set[:5]:
        try:
            try:
                resp_info = requests.get("http://api.elsevier.com/content/article/"+identifier+"?&view=FULL",
                                headers={'Accept':'application/json',
                                         'X-ELS-APIKey': APIkey})
                results_info = resp_info.json()
                _ = results_info["full-text-retrieval-response"]["coredata"]["dc:title"]
            except:
                resp_info = requests.get("http://api.elsevier.com/content/article/"+identifier,
                    headers={'Accept':'application/json',
                         'X-ELS-APIKey': APIkey})
                results_info = resp_info.json()
                _ = results_info["full-text-retrieval-response"]["coredata"]["dc:title"]



            if (idx+start)%1==0:
                print(idx+start,"papers are extracted")

            ### number of citation ###
            try:
                resp_c = requests.get("http://api.elsevier.com/content/search/index:SCOPUS?query=DOI("+identifier[4:]+")&field=citedby-count",
                        headers={'X-ELS-APIKey': APIkey})
                results_c = resp_c.json()
                cv = results_c['search-results']['entry']
                cc = cv[0]['citedby-count']
            except:
                cc = 0

            ### title ###
            try:        
                title=results_info["full-text-retrieval-response"]["coredata"]["dc:title"]
            except:
                title="not provided"

            ### date of publication ###
            try:        
                date=results_info["full-text-retrieval-response"]["coredata"]["prism:coverDate"]
            except:
                date="not provided"

            ### authors ###    
            try:
                authors=[]
                for x in range(0,len(results_info["full-text-retrieval-response"]["coredata"]['dc:creator'])):
                    authors.append(results_info["full-text-retrieval-response"]["coredata"]['dc:creator'][x]['$'])
            except:
                authors="not provided"

            ### Author provided keywords ###
            try:
                apk=[]
                for x in range(0,len(results_info["full-text-retrieval-response"]["coredata"]['dcterms:subject'])):
                    apk.append(results_info["full-text-retrieval-response"]["coredata"]['dcterms:subject'][x]['$'])
            except:
                apk="not provided"

            ### journal/book name ###
            try:        
                journal=results_info["full-text-retrieval-response"]["coredata"]['prism:publicationName']
            except:
                journal="not provided"

            ### type of publication ###
            try:        
                jtype=results_info["full-text-retrieval-response"]["coredata"]['prism:aggregationType']
            except:
                jtype="not provided"

            ### abstract ###    
            try:        
                abst=results_info["full-text-retrieval-response"]["coredata"]['dc:description']
            except:
                abst="not provided"

            # links for the paper
            try:
                links = results_info['full-text-retrieval-response']['coredata']['prism:url']
            except:
                links = "not provided"

            ### full body ###
            try:
                full_body = results_info['full-text-retrieval-response']['originalText'].split(results_info['full-text-retrieval-response']['coredata']['dc:description'])[-1]
            except:
                full_body = "not provided"
            f = open(current_time+'/'+str(idx+start)+'.txt','w')
            f.write(full_body)
            f.close()


            intextdata=intextdata.append({'DOI':identifier,'title':title,'date':date,'authors':authors,
                    'keywords':apk,'journal_or_book_name':journal,
                    'journal_type':jtype,'abstract':abst,"citation_count":int(cc),"paper_id":int(idx+start),"link":links},ignore_index=True)
            idx+=1
        except:
            print(idx,": full review failed")
    
    return intextdata,num_results

In [7]:
textdata = extract_metadata(keywords,APIkey)

100.0 % of results from 1 ~ 100 can be accessed
1 papers are extracted
2 papers are extracted
3 papers are extracted
4 papers are extracted
5 papers are extracted


In [10]:
textdata.head()

,DOI,abstract,authors,citation_count,date,journal_or_book_name,journal_type,keywords,link,paper_id,title
0,DOI:10.1016/j.ymssp.2018.02.016,\n Abstract\n \n ...,"[Liu, Ruonan, Yang, Boyuan, Zio, Enrico, Chen,...",0.0,2018-08-31,Mechanical Systems and Signal Processing,Journal,"[Artificial intelligence, Fault diagnosis, \n ...",http://api.elsevier.com/content/article/pii/S0...,1.0,Artificial intelligence for fault diagnosis of...
1,DOI:10.1016/j.osn.2017.12.006,Abstract Artificial intelligence (AI) is an ex...,"[Mata, Javier, de Miguel, Ignacio, Durán, Ramó...",0.0,2018-04-30,Optical Switching and Networking,Journal,"[Artificial intelligence, Machine learning, Op...",http://api.elsevier.com/content/article/pii/S1...,2.0,Artificial intelligence (AI) methods in optica...
2,DOI:10.1016/j.engappai.2017.12.011,\n Abstract\n \n ...,"[Elmishali, Amir, Stern, Roni, Kalech, Meir]",0.0,2018-03-31,Engineering Applications of Artificial Intelli...,Journal,"[Artificial Intelligence, Automated diagnosis,...",http://api.elsevier.com/content/article/pii/S0...,3.0,An Artificial Intelligence paradigm for troubl...
3,DOI:10.1016/j.clinimag.2017.11.007,\n Abstract\n \n ...,"[Mayo, Ray Cody, Leung, Jessica]",0.0,2018-06-30,Clinical Imaging,Journal,"[Artificial intelligence, Deep learning, Big d...",http://api.elsevier.com/content/article/pii/S0...,4.0,Artificial intelligence and deep learning – Ra...
4,DOI:10.1016/j.eap.2017.12.006,\n Abstract\n \n ...,"[Fujii, Hidemichi, Managi, Shunsuke]",0.0,2018-06-30,Economic Analysis and Policy,Journal,"[Artificial intelligence, Patent decomposition...",http://api.elsevier.com/content/article/pii/S0...,5.0,Trends and priority shifts in artificial intel...


In [9]:
pickle.dump(textdata,open( current_time+'/extracted_data.p', "wb" ))